In [1]:
%%capture
%run test_data.ipynb

In [14]:
p10001 = data.loc[data['PERMNO'] == 10001][['date','RET']].set_index('date')
p10001.columns = [10001]

p93436 = data.loc[data['PERMNO'] == 93436][['date','RET']].set_index('date')
p93436.columns = [93436]

In [53]:
import multiprocessing as mp
from multiprocessing.pool import ThreadPool

def threadify(f, arr):
    # Maps array with function f parallelized
    pool = ThreadPool(mp.cpu_count())
    result = pool.map(f,arr)
    pool.terminate()
    pool.join()
    return result

def permno_to_timeseries(data,permno):
    # Helper for crsp_to_timeseries
    df = data.loc[data['PERMNO'] == permno][['date','RET']].set_index('date')
    df.columns = [permno]
    return df

def crsp_to_timeseries(data):
    # Return timeseries DataFrame of unique permnos
    permnos = data['PERMNO'].unique()
    dfs = threadify(lambda e: permno_to_timeseries(data,e),permnos)
    dfs = threadify(lambda df: df.loc[~df.index.duplicated()],dfs)
    return pd.concat(dfs,axis=1,sort=False)

In [55]:
timeser = crsp_to_timeseries(data)

In [74]:
timeser[[10001,93436]].cov()

,10001,93436
10001,0.014999,-0.001229
93436,-0.001229,0.015010


In [79]:
covm = timeser[[93418,93420,93422,93423,93426]].cov().to_numpy()

In [82]:
covm

array([[0.03617916, 0.01245388, 0.00307017, 0.00108404, 0.00077745],
       [0.01245388, 0.03858536, 0.02402099, 0.00333742, 0.00661527],
       [0.00307017, 0.02402099, 0.02690761, 0.0033782 , 0.00490849],
       [0.00108404, 0.00333742, 0.0033782 , 0.00576125, 0.00075839],
       [0.00077745, 0.00661527, 0.00490849, 0.00075839, 0.01051711]])

In [75]:
timeser[10001].var()

0.014998628314980647

In [76]:
from scipy.optimize import minimize

In [95]:
t = lambda X: np.transpose(X)
mm = lambda A,B: np.matmul(A,B)

pvar = lambda w: mm(mm(t(w),covm),w)

In [96]:
w = np.array([0.2,0.2,0.2,0.2,0.2])
pvar(w)

0.009550363491191689

In [125]:
#sum_to_one = [{'type': 'eq', 'fun': lambda w:  np.sum(w)-1.0}]

#cons = [{'type': 'ineq', 'fun': lambda w: w[i]} for i in range(len(w))] + sum_to_one

cons = [{'type': 'eq', 'fun': lambda w:  np.sum(w)-1.0}]
b = [(0, None) for i in range(len(w))]

In [126]:
res = minimize(pvar, w, method='SLSQP',constraints=cons, bounds=b)

In [127]:
sum(res.x)

1.0

In [129]:
print(res)

     fun: 0.0038174434229773053
     jac: array([0.00782195, 0.01021806, 0.00765065, 0.00765256, 0.00754971])
 message: 'Optimization terminated successfully.'
    nfev: 105
     nit: 15
    njev: 15
  status: 0
 success: True
       x: array([0.08324941, 0.        , 0.        , 0.60780824, 0.30894235])


In [130]:
res.x

array([0.08324941, 0.        , 0.        , 0.60780824, 0.30894235])

In [131]:
res.success

True

In [133]:
np.array([1,1,1,1])/sum(np.array([1,1,1,1]))

array([0.25, 0.25, 0.25, 0.25])

In [ ]:
def optimal_weights(covm):
    t = lambda X: np.transpose(X)
    mm = lambda A,B: np.matmul(A,B)
    pvar = lambda w: mm(mm(t(w),covm),w)
    
    # Weights sum to 1
    cons = [{'type': 'eq', 'fun': lambda w:  np.sum(w)-1.0}]
    # No short sales
    b = [(0, None) for i in range(len(w))]
    
    res = minimize(pvar, w, method='SLSQP',constraints=cons, bounds=b)
    
    # Returns minimization successful return optimal weights
    # Else return equal weights
    if res.success:
        return res.x
    else:
        